In [1]:
import os
import math
import pandas as pd
import numpy as np
import urllib
import matplotlib.pyplot as plt
%matplotlib inline

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

import datetime
import dateparser

import hydrofunctions as hf

import fbprophet 
from fbprophet.diagnostics import cross_validation

# For inputting City information to get distance to sensors.
from  geopy.geocoders import Nominatim
import geopy.distance

# Date time conversion registration.
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()


In [2]:
## Reading in site data from 'Gage_sites.txt'
file = 'Gage_sites.txt'
sites = pd.read_csv(file, sep = '\t', header = 39, skiprows = [40], dtype = {"site_no" : "str"})

##Removes sites with no data for past 20 years. (12 sites)
sites = sites.drop([65, 71, 77, 117, 118, 138, 203, 225, 320, 330, 331, 338])
sites = sites.reset_index(drop = True)

##Removes data that is unable to be cross validated due to too few observations. (25 sites)
sites = sites.drop([18, 83, 84, 85, 89, 92, 101, 103, 104, 136, 138, 146, 148, 149, 150, 151, 179, 208, 209, 224, 228, 231, 236, 293, 314])
sites = sites.reset_index(drop = True)

site_no = list(sites["site_no"])
site_nm = list(sites["station_nm"])
site_loc = sites.filter(['site_no', 'station_nm', 'dec_lat_va', 'dec_long_va'])

In [3]:
username = 'cadeadams'

dbname = 'usgs_stream_db_live'
dbname2 = 'usgs_stream_db'
dbname3 = 'usgs_stream_db_log'

engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
engine2 = create_engine('postgres://%s@localhost/%s'%(username,dbname2))
engine3 = create_engine('postgres://%s@localhost/%s'%(username,dbname3))

if not database_exists(engine.url):
    create_database(engine.url)

if not database_exists(engine2.url):
    create_database(engine2.url)

if not database_exists(engine3.url):
    create_database(engine3.url)

In [4]:
#site_no.index('394220106431500') ## Has water temp data. Need to extract...

site_loc.to_sql('site_locations', engine3, if_exists='replace')

In [9]:
##
# Pulling in data using hydrofunctions and saving to PostgreSQL database.
##
start = '2000-01-01'
end = str(datetime.datetime.today().strftime('%Y-%m-%d')) #Gets today's date.

for site in site_no :
    herring = hf.NWIS(site, 'dv', start, end)
    herring.get_data()
    my_dict = herring.json()
    df = hf.extract_nwis_df(my_dict)
    df.rename(index=str, columns = {"USGS:"+site+":00060:00003" : "y", 
                                    "USGS:"+site+":00060:00003_qualifiers" : "flags"}, 
             inplace = True)
#    df['ds'] = df.index[:]
#    df['ds'].str.split(pat = ' ', expand = True)
    df.to_sql("n"+str(site), engine3, if_exists='replace')

In [254]:
con = None
con = psycopg2.connect(database = dbname3, user = username)

sql_query = """
SELECT * FROM n06730500;
"""

site_data_from_sql = pd.read_sql_query(sql_query,con)

df_site = site_data_from_sql
df_site = df_site.rename(columns={'datetime':'ds'})
df_site.head()

,ds,y,flags
0,2000-01-01 00:00:00,37.0,A
1,2000-01-02 00:00:00,37.0,A
2,2000-01-03 00:00:00,38.0,A
3,2000-01-04 00:00:00,35.0,A
4,2000-01-05 00:00:00,35.0,A


In [255]:
x = pd.to_datetime(df_site['ds'])

fig, ax1 = plt.subplots(figsize=(15,10))
ax1.plot(x, df_site.y, color='blue')
ax1.set_title('Daily Mean Discharge (2018-2020)')
ax1.set_ylabel('Discharge (cubic feet per second)')
ax1.set_xlabel('Date')
ax1.set_xlim(pd.Timestamp('2018-01-01'), pd.Timestamp('2020-01-01'))
ax1.set_yscale('log')
fig.savefig('test1.png')
plt.close(fig)

#pd.DataFrame(df_site.set_index('ds').y).plot()
#plt.yscale('log')
##plt.xlim(pd.Timestamp('2017-01-01'), pd.Timestamp('2021-01-01'))
#plt.savefig('test1.png')
#plt.close()

/Users/cadeadams/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



In [256]:
df_site['y'] = np.log(df_site['y'])

pd.DataFrame(df_site.set_index('ds').y).plot()
plt.yscale('log')
#plt.xlim(pd.Timestamp('2017-01-01'), pd.Timestamp('2021-01-01'))
plt.savefig('test2.png')
plt.close()

In [257]:
df_prophet = fbprophet.Prophet(changepoint_prior_scale=0.05, yearly_seasonality=True, interval_width = 0.25)
df_prophet.fit(df_site)

df_forecast = df_prophet.make_future_dataframe(periods=450 * 1, freq='D')
df_forecast = df_prophet.predict(df_forecast)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [258]:
fig = df_prophet.plot(df_forecast)
ax = fig.gca()
ax.set_ylabel('ln[Discharge (cubic feet per second)]')
ax.set_xlabel('Date')
ax.set_xlim(pd.Timestamp('2018-01-01'), pd.Timestamp('2021-01-01'))
fig.savefig('test_mod.png');

In [259]:
df_site.set_index('ds', inplace=True)
df_forecast.set_index('ds', inplace=True)

site_data_from_sql = pd.DataFrame(site_data_from_sql)

site_data_from_sql.set_index('datetime', inplace=True)

viz_df = site_data_from_sql.join(df_forecast[['yhat', 'yhat_lower','yhat_upper']], how = 'outer')

viz_df['yhat_rescaled'] = np.exp(viz_df['yhat'])

#viz_df[['y', 'yhat_rescaled']].plot()
#plt.savefig('test_comp.png')

In [260]:
viz_df[['y', 'yhat_rescaled']].plot()
plt.xlim(pd.Timestamp('2018-01-01'), pd.Timestamp('2020-01-01'))
plt.xlabel('Date')
plt.yscale('log')
plt.ylabel('Dicharge (cubic feet per second) [log scale]')
plt.fill_between(viz_df.index, np.exp(viz_df['yhat_upper']), np.exp(viz_df['yhat_lower']), alpha=0.5, color='darkgray')
plt.savefig('test_comp.png')
plt.close()

In [261]:
## Only to showing forcasted values plus errors

site_data_from_sql.index = pd.to_datetime(site_data_from_sql.index)
connect_date = site_data_from_sql.index[-2]

mask = (df_forecast.index > connect_date)
predict_df = df_forecast.loc[mask]

viz_df = site_data_from_sql.join(predict_df[['yhat', 'yhat_lower','yhat_upper']], how = 'outer')
viz_df['yhat_scaled']=np.exp(viz_df['yhat'])

In [262]:
fig, ax1 = plt.subplots(figsize=(15,10))
ax1.plot(viz_df.y, color='blue')
ax1.plot(viz_df.yhat_scaled, color='black', linestyle=':')
ax1.fill_between(viz_df.index, np.exp(viz_df['yhat_upper']), np.exp(viz_df['yhat_lower']), alpha=0.5, color='darkgray')
ax1.set_title('Discharge (Blue) vs Discharge Forecast (Black)')
ax1.set_ylabel('Discharge (cubic feet per second)')
ax1.set_xlabel('Date')
ax1.set_xlim(pd.Timestamp('2018-01-01'), pd.Timestamp('2021-01-01'))
ax1.set_yscale('log')

#L=ax1.legend() #get the legend
#L.get_texts()[0].set_text('Actual Discharge') #change the legend text for 1st plot
#L.get_texts()[1].set_text('Forecasted Discharge') #change the legend text for 2nd plot

fig.savefig('test_final.png')
plt.close(fig)

/Users/cadeadams/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).



# Setting up Forecasts on log data to ensure non-zero/negative flow rates. Converting back to original scale in the final forecast dataframe save to SQL.

In [16]:
##
# Modeling data using fbprophet and saving to PostgreSQL database.
##
con = None
con = psycopg2.connect(database = dbname3, user = username)

for site in site_no :
    sql_query = """
    SELECT * FROM n"""+site+""";
    """
    site_data_from_sql = pd.read_sql_query(sql_query,con)
    nonzero_mean = site_data_from_sql[ site_data_from_sql.y != 0 ].mean()
    site_data_from_sql.loc[ site_data_from_sql.y == 0, "y" ] = nonzero_mean

    df_site = site_data_from_sql
    df_site = df_site.rename(columns={'datetime':'ds'})
    df_site['y'] = np.log(df_site['y'])
    
    df_prophet = fbprophet.Prophet(changepoint_prior_scale=0.05, yearly_seasonality=True, interval_width = 0.75)
    df_prophet.fit(df_site)
    
    df_forecast = df_prophet.make_future_dataframe(periods=450 * 1, freq='D')
    df_forecast = df_prophet.predict(df_forecast)
    
    df_site.set_index('ds', inplace=True)
    df_forecast.set_index('ds', inplace=True)

    site_data_from_sql = pd.DataFrame(site_data_from_sql)

    site_data_from_sql.set_index('datetime', inplace=True)

    viz_df = site_data_from_sql.join(df_forecast[['yhat', 'yhat_lower','yhat_upper']], how = 'outer')

    viz_df['ds'] = viz_df.index
    viz_df['yhat_rescaled'] = np.exp(viz_df['yhat'])
    viz_df['yhat_upper_rescaled'] = np.exp(viz_df['yhat_upper'])
    viz_df['yhat_lower_rescaled'] = np.exp(viz_df['yhat_lower'])

    viz_df.to_sql("n"+str(site)+"_forecast", engine3, if_exists='replace')

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=Tru

In [10]:
print(site)

06701700


In [15]:
con = None
con = psycopg2.connect(database = dbname3, user = username)

sql_query = """
SELECT * FROM n06701700;
"""

query_results = pd.read_sql_query(sql_query,con)
print(query_results.describe())
nonzero_mean = query_results[ query_results.y != 0 ].mean()
query_results.loc[ query_results.y == 0, "y" ] = nonzero_mean
print(query_results.describe())

                 y
count  1038.000000
mean      7.836522
std       9.176524
min       0.000000
25%       2.622500
50%       5.305000
75%       8.637500
max      58.000000
                 y
count  1024.000000
mean      7.943662
std       9.192883
min       0.210000
25%       2.660000
50%       5.470000
75%       8.725000
max      58.000000


In [60]:
con = None
con = psycopg2.connect(database = dbname, user = username)

sql_query = """
SELECT * FROM site_locations;
"""

query_results = pd.read_sql_query(sql_query,con)

location = [38.7511041, -105.521384]
date = "Sep 23, 2020"
t = dateparser.parse(date)

site_no = query_results["site_no"]
site_lat = query_results["dec_lat_va"]
site_long = query_results["dec_long_va"]

sites_coord = pd.DataFrame([site_no, site_lat, site_long])
sites_coord = sites_coord.T

distance = []
for i in range(len(sites_coord)) :
    distance.append(geopy.distance.distance(location, sites_coord.iloc[i,1:]).miles)

query_results["distance"] = distance
site_no = pd.DataFrame(site_no)
site_no["distance"] = distance
query_results = query_results.sort_values(by = ["distance"])
site_no = site_no.sort_values(by = ["distance"])

count = 0
loc_lat = list()
loc_lon = list()
flow = list()
flow_upper = list()
flow_lower = list()
good_site = list()
good_dist = list()

for i in range(len(site_no)) :
    sql_query_model = """
                      SELECT * FROM n"""+site_no['site_no'].iloc[i]+"""_forecast;
                      """
    query_results_model = pd.read_sql_query(sql_query_model,con)
    if (t == query_results_model['ds']).any() :
        temp = query_results_model.loc[query_results_model['ds'] == t]
        if (temp['yhat'].iloc[0] > 100 and temp['yhat'].iloc[0] < 400) :
            loc_lat.append(float(query_results[i:i+1]["dec_lat_va"]))
            loc_lon.append(float(query_results[i:i+1]["dec_long_va"]))
            good_site.append(query_results[i:i+1]["site_no"].iloc[0])
            good_dist.append(query_results[i:i+1]["distance"].iloc[0])
            flow.append(temp['yhat'].iloc[0])
            flow_upper.append(temp['yhat_upper'].iloc[0])
            flow_lower.append(temp['yhat_lower'].iloc[0])
            count = count + 1
            if count == 3 :
                break
        else :
            continue

#site_data_from_sql = pd.read_sql_query(sql_query_dat,con)
#site_model_from_sql = pd.read_sql_query(sql_query_mod,con)

[230.41453120574732, 299.12043536370714, 423.05635695726676]
[174.5370202258796, 253.48735318298114, 309.8214353732134]
[129.77860381352696, 205.5372636417656, 154.6292389192572]


In [20]:
con = None
con = psycopg2.connect(database = dbname3, user = username)

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100


mape_baseline = list()
for site in site_no :
    sql_query = """
                SELECT * FROM n"""+site+""";
                """
    site_data_from_sql = pd.read_sql_query(sql_query,con)

    df_prophet = fbprophet.Prophet(changepoint_prior_scale=0.05, daily_seasonality=True, interval_width = 0.75)
    df_prophet.fit(site_data_from_sql)

    cv_results = cross_validation(df_prophet, initial = '1095 days', period = '180 days', horizon = '365 days')
    cv_results.to_sql("n"+str(site)+"_cv", engine3, if_exists='replace')
    mape_baseline.append(mean_absolute_percentage_error(cv_results.y, cv_results.yhat))


INFO:fbprophet:Making 26 forecasts with cutoffs between 2006-10-02 00:00:00 and 2019-01-27 00:00:00
/Users/cadeadams/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning:

divide by zero encountered in true_divide

INFO:fbprophet:Making 3 forecasts with cutoffs between 2018-01-31 00:00:00 and 2019-01-26 00:00:00
INFO:fbprophet:Making 33 forecasts with cutoffs between 2003-04-20 00:00:00 and 2019-01-26 00:00:00
INFO:fbprophet:Making 28 forecasts with cutoffs between 2005-10-06 00:00:00 and 2019-01-26 00:00:00
INFO:fbprophet:Making 33 forecasts with cutoffs between 2003-04-20 00:00:00 and 2019-01-26 00:00:00
INFO:fbprophet:Making 19 forecasts with cutoffs between 2010-01-18 00:00:00 and 2018-12-02 00:00:00
INFO:fbprophet:Making 33 forecasts with cutoffs between 2003-04-20 00:00:00 and 2019-01-26 00:00:00
INFO:fbprophet:Making 20 forecasts with cutoffs between 2009-07-24 00:00:00 and 2018-12-04 00:00:00
INFO:fbprophet:Making 8 forecasts with cutoffs between 2015-0

In [49]:
cross_val_all = pd.DataFrame()

file = 'Gage_sites.txt'

sites_cv = pd.read_csv(file, sep = '\t', header = 39, skiprows = [40], dtype = {"site_no" : "str"})

##Removes sites with no data for past 20 years.
sites_cv = sites_cv.drop([65, 71, 77, 117, 118, 138, 203, 225, 320, 330, 331, 338])
sites_cv = sites_cv.reset_index(drop = True)

##Removes data that is unable to be cross validated due to too few observations.
sites_cv = sites_cv.drop([18, 83, 84, 85, 89, 92, 101, 103, 104, 136, 138, 146, 148, 149, 150, 151, 179, 208, 209, 224, 228, 231, 236, 293, 314])
sites_cv = sites_cv.reset_index(drop = True)

site_no_cv = list(sites_cv["site_no"])
site_nm_cv = list(sites_cv["station_nm"])
site_ele_cv = list(sites_cv["alt_va"])

cross_val_all["site_no"] = site_no_cv

mape_baseline = list()
for site in site_no_cv :
    sql_query = """
                SELECT * FROM n"""+site+"""_cv;
                """
    site_cv_from_sql = pd.read_sql_query(sql_query,con)
    mape_baseline.append(mean_absolute_percentage_error(site_cv_from_sql.y, site_cv_from_sql.yhat))

cross_val_all["mape_baseline"] = mape_baseline
cross_val_all["elevation"] = site_ele_cv

/Users/cadeadams/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning:

divide by zero encountered in true_divide

/Users/cadeadams/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning:

invalid value encountered in true_divide



    site_no  mape_baseline  elevation
0  06614800     191.438191   10390.00
1  06620000     180.378014    7810.39
2  06696980     114.760852    9935.00
3  06700000      70.095801    6845.86
4  06701620     236.782235    7440.00


In [52]:
plt.scatter(cross_val_all["elevation"], cross_val_all["mape_baseline"])
plt.xscale("log")
plt.yscale("log")
plt.savefig("cv_eda.png")
plt.close()

In [61]:
cross_val_nona = cross_val_all.replace([np.inf, -np.inf], np.nan)
print(cross_val_nona["mape_baseline"].describe())
print(cross_val_nona.isna().sum())

count     245.000000
mean      264.221824
std       367.922762
min        37.889536
25%       104.716382
50%       163.797031
75%       265.845234
max      4015.169617
Name: mape_baseline, dtype: float64
site_no           0
mape_baseline    57
elevation         1
dtype: int64


# Sites removed during validation phase
n06711040 (207 days only) 18
n07103784 (257 days only) 83
n07103791 (257 days only) 84
n07103792 (471 days only) 85
n07103965 (676 days only) 89
n07103987 (646 days only) 92
n07105780 (814 days only) 101
n07105815 (868 days only) 103
n07105820 (867 days only) 104
n09019000 (405 days only) 136
n09021000 (677 days only) 138
n09032050 (288 days only) 146
n09032200 (295 days only) 148
n09032300 (304 days only) 149
n09032990 (337 days only) 150
n09033010 (309 days only) 151
n09060799 (867 days only) 179
n09076000 (757 days only) 208
n09076300 (575 days only) 209
n09110990 (447 days only) 224
n09113500 (245 days only) 228
n09114520 (459 days only) 231
n09124010 (188 days only) 236
n09352800 (647 days only) 293
n383103106594200 (803 days only) 314

In [36]:
df_forecast = df_prophet.make_future_dataframe(periods=365 * 1, freq='D')
df_forecast = df_prophet.predict(df_forecast)
df_prophet.plot(df_forecast)
plt.xlabel('Date')
plt.yscale('log')
plt.ylabel('Discharge (cfs)')
plt.savefig('test.png')
plt.close()

In [46]:
#type(pd.to_datetime(site_model_from_sql['ds']))
date = "Sep 23, 2020"
t = dateparser.parse(date)
df_forecast.loc[df_forecast['ds'] == t]

             ds      trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
7571 2020-09-23  80.014778   47.740213  108.163608    80.014778    80.014778   

      additive_terms  additive_terms_lower  additive_terms_upper     daily  \
7571        9.691244              9.691244              9.691244 -5.712846   

      ...    weekly  weekly_lower  weekly_upper     yearly  yearly_lower  \
7571  ... -5.572282     -5.572282     -5.572282  20.976372     20.976372   

      yearly_upper  multiplicative_terms  multiplicative_terms_lower  \
7571     20.976372                   0.0                         0.0   

      multiplicative_terms_upper       yhat  
7571                         0.0  89.706022  

[1 rows x 22 columns]


# Everything below here is for collection and placing into PostgreSQL database. FBProphet models are also performed over all data collected.

In [16]:
df.plot()
plt.xlabel('Date')
plt.yscale('log')
plt.ylabel('Discharge (cfs)')
plt.title('Daily Mean Discharge for Roaring Fork River')
plt.savefig('RRFork1.png')
plt.close()

In [9]:
df_prophet = fbprophet.Prophet(changepoint_prior_scale=0.05, daily_seasonality=True, interval_width = 0.25)
df_prophet.fit(df)

In [10]:
df_forecast = df_prophet.make_future_dataframe(periods=365 * 1, freq='D')
df_forecast = df_prophet.predict(df_forecast)

In [12]:
df_prophet.plot_components(df_forecast)
plt.savefig('RRFork1_Components.png')
plt.close()

In [83]:
site_data_from_sql.head()

,datetime,y,flags,ds,y_orig,y_log
0,2000-01-01 00:00:00,67.0,A,2000-01-01 00:00:00,67.0,1.826075
1,2000-01-02 00:00:00,81.0,A,2000-01-02 00:00:00,81.0,1.908485
2,2000-01-03 00:00:00,76.0,A,2000-01-03 00:00:00,76.0,1.880814
3,2000-01-04 00:00:00,102.0,A,2000-01-04 00:00:00,102.0,2.008600
4,2000-01-05 00:00:00,79.0,A,2000-01-05 00:00:00,79.0,1.897627


In [62]:
con = None
con = psycopg2.connect(database = dbname, user = username)

sql_query_fore = """
                 SELECT * FROM n06730500_forecast;
                 """

query_results_plt_fore = pd.read_sql_query(sql_query_fore,con)

sql_query = """
            SELECT * FROM n06730500;
            """

query_results_plt = pd.read_sql_query(sql_query,con)


query_results_plt.tail()

,datetime,y,flags,ds
7333,2020-01-29 00:00:00,49.8,P,2020-01-29 00:00:00
7334,2020-01-30 00:00:00,49.5,P,2020-01-30 00:00:00
7335,2020-01-31 00:00:00,49.2,P,2020-01-31 00:00:00
7336,2020-02-01 00:00:00,47.7,P,2020-02-01 00:00:00
7337,2020-02-02 00:00:00,48.5,P,2020-02-02 00:00:00


In [63]:
query_results_plt['ds'] = pd.to_datetime(query_results_plt['ds'])
query_results_plt_fore['ds'] = pd.to_datetime(query_results_plt_fore['ds'])

ax = query_results_plt.plot(x = 'ds', y = 'y')
query_results_plt_fore.plot(x = 'ds', y = 'yhat', ax = ax)
ax.set_xlim(pd.Timestamp('2017-01-01'), pd.Timestamp('2020-01-01'))
#ax.set_yscale('log')
fig = ax.get_figure()
fig.savefig('test.png')
plt.close(fig)

In [64]:
query_results_plt_fore[query_results_plt_fore['ds'] >= pd.Timestamp('2018-07-01')].describe()

,index,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
count,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.000000,1032.0,1032.0,1032.0,1032.000000
mean,7271.500000,65.365889,-0.284832,116.155053,65.362571,65.372399,-7.888672,-7.888672,-7.888672,-0.000715,-0.000715,-0.000715,-7.887957,-7.887957,-7.887957,0.0,0.0,0.0,57.477217
std,298.057042,8.904553,59.761838,59.640899,8.910280,8.893593,58.656185,58.656185,58.656185,3.268613,3.268613,3.268613,58.566207,58.566207,58.566207,0.0,0.0,0.0,59.161959
min,6756.000000,49.965155,-99.548741,13.000407,49.586589,50.426975,-94.647562,-94.647562,-94.647562,-5.565332,-5.565332,-5.565332,-89.153903,-89.153903,-89.153903,0.0,0.0,0.0,-37.141952
25%,7013.750000,57.665522,-31.823901,84.232674,57.665522,57.665522,-33.552571,-33.552571,-33.552571,-3.422321,-3.422321,-3.422321,-32.833959,-32.833959,-32.833959,0.0,0.0,0.0,26.381858
50%,7271.500000,65.365889,-17.736133,98.430991,65.365889,65.365889,-26.135698,-26.135698,-26.135698,0.816370,0.816370,0.816370,-25.740097,-25.740097,-25.740097,0.0,0.0,0.0,39.751780
75%,7529.250000,73.066255,3.539787,118.829535,73.066255,73.066255,-7.222516,-7.222516,-7.222516,3.103535,3.103535,3.103535,-10.101176,-10.101176,-10.101176,0.0,0.0,0.0,59.088802
max,7787.000000,80.766622,221.773563,337.780254,80.766622,80.766622,199.860249,199.860249,199.860249,4.171851,4.171851,4.171851,195.727278,195.727278,195.727278,0.0,0.0,0.0,270.439383


In [265]:
file = 'bouldercreekIV.csv'
cc_iv = pd.read_csv(file, sep = '\t', header = 27, skiprows = [28], dtype = {"site_no" : "str"})
cc_iv.rename(columns = {"211034_00060" : "y", "211034_00060_cd" : "flags"}, inplace=True)

x = pd.to_datetime(cc_iv['datetime'])

fig, ax1 = plt.subplots(figsize=(15,10))
ax1.plot(x, cc_iv.y, color='blue')
ax1.set_title('Instantaneous Discharge (Jan 2, 2020 - Feb 9, 2020)')
ax1.set_ylabel('Discharge (cubic feet per second)')
ax1.set_xlabel('Date')
ax1.set_xlim(pd.Timestamp('2020-01-02'), pd.Timestamp('2020-02-10'))
ax1.set_yscale('log')
fig.savefig('test_IV.png')
plt.close(fig)

/Users/cadeadams/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning:

More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).

